In [23]:
!pip install pytictoc --upgrade

In [24]:
import numpy as np
from pytictoc import TicToc

In [25]:
def is_poitive_definite(A):
    try:
        np.linalg.cholesky(A)
    except np.linalg.LinAlgError:
        return False
    return True



In [26]:
def hybrid_newton(f,gf,hf,lsearch,x0,eps):
    clock = TicToc()
    clock.tic()
    xk = x0
    fs = [x0]
    ts = [0]
    gs = [gf(x0)]
    newton = []
    while np.linalg.norm(gf(xk)) >= eps:
        if is_poitive_definite(hf(xk)):
            dk = (np.linalg.solve(hf(xk), -gf(xk)),'newton')
            newton.append(1)
        else:
            dk = (-gf(xk),'grad')
            newton.append(0)
        grad_xk= gf(xk)
        tk = lsearch(xk,grad_xk,dk)
        xk = xk - tk*dk[0]
        fs.append(f(xk))
        gs.append(np.linalg.norm(gf(xk)))
        ts.append(clock.tocvalue())
    return x, fs,gs,ts,newton


In [27]:
def calc_t(f, xk, gk, dk, alpha, beta, s):
    t = s
    # print(b - (A.dot(xk - t * dk)))
    while f(xk + t * dk) >= (f(xk) + (alpha * t * (gk(xk).T.dot(dk)))):
        t = beta * t
    return t

In [28]:
def hybrid_back(f,alpha,beta,s):
    return lambda xk,gk,direction: calc_t(f,xk,gk,direction[0],alpha,beta,s) if direction[1]=='grad' else 1


In [35]:
f = lambda x: x[0]**4 + x[1]**4 - 36*x[0]*x[1]
gf = lambda x: np.array([4*(x[0]**3) - 36*x[1],4*(x[1]**3) - 36*x[0]])
hf = lambda x: np.array([[12*x[0]**2 -36, -36],[-36,12*x[1]**2 -36]])
lsearch = hybrid_back(f,alpha=0.25,beta=0.5,s=1)
x0 = np.array([200,0])
eps = pow(10,-6)
x,fs,gs,ts,newton = hybrid_newton(f,gf,hf,lsearch,x0,eps)

<ipython-input-35-9d11e3ce0557>:1: RuntimeWarning: overflow encountered in scalar multiply
  f = lambda x: x[0]**4 + x[1]**4 - 36*x[0]*x[1]


TypeError: 'numpy.ndarray' object is not callable

Traceback (most recent call last):
  File "_pydevd_bundle\pydevd_cython_win32_38_64.pyx", line 1034, in _pydevd_bundle.pydevd_cython_win32_38_64.PyDBFrame.trace_dispatch
  File "C:\Program Files\JetBrains\PyCharm 2022.1\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "C:\Program Files\JetBrains\PyCharm 2022.1\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "C:\Program Files\JetBrains\PyCharm 2022.1\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined
